# PaleoClimate Plot Data

In [1]:
import geopandas as gpd
import folium
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier


### The dataset with information about paleoclimate, collected by research geologists: 
   - Age - Geological age of evidence. 
   - lat, long - The geographic coordinates of the evidence. 
   - basin - Belonging river basin. 
   - Climate - Climate reference

In [2]:
climate_data = pd.read_csv('dataset/points.csv')
climate_data.head()


,age,basin,lat,long,climate
0,145,camamu-almada,-13.83,-38.5,h
1,145,camamu-almada,-13.83,-38.5,h
2,145,camamu-almada,-13.83,-38.5,h
3,140,araripe,-7.50,-39.5,h
4,140,Camamu-almada,-13.83,-38.5,h


## Apply de dataset points into map


In [18]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

# Ler dados de clima a partir do arquivo CSV
climate_data = pd.read_csv('dataset/points.csv')

# Mapeamento de climas
climate_mapping = {
    'h': 1,  # Exemplo: "h" para 1
    's': 2,  # Exemplo: "s" para 2
    't': 3   # Exemplo: "t" para 3
}

# Aplicando mapeamento aos dados de clima
climate_data['climate'] = climate_data['climate'].map(climate_mapping)

# Remover linhas com NaN na coluna 'climate'
climate_data = climate_data.dropna(subset=['climate'])

# Separando features (X) e alvo (y)
X = climate_data[['lat', 'long', 'age']]
y = climate_data['climate']

# Normalizando os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Treinando o modelo KNN (k=3)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_scaled, y)

# Função para prever o clima em uma nova posição
def prever_clima(lat, long, age):
    ponto_hipotetico = pd.DataFrame({
        'lat': [lat],
        'long': [long],
        'age': [age]
    })

    # Normalizando o ponto
    ponto_hipotetico_scaled = scaler.transform(ponto_hipotetico)

    # Fazendo a previsão
    clima_predito = knn.predict(ponto_hipotetico_scaled)
    return clima_predito[0]

# Mapeamento inverso para converter números de volta para string
inverse_climate_mapping = {v: k for k, v in climate_mapping.items()}

# Ler o CSV com os pontos a serem previstos
points_to_predict = pd.read_csv('dataset/points_to_predict.csv')

# Aplicar previsões em cada linha do CSV
points_to_predict['predicted_climate'] = points_to_predict.apply(
    lambda row: prever_clima(row['lat'], row['long'], row['age']), axis=1
)

# Converter valores numéricos de volta para strings de clima
points_to_predict['predicted_climate'] = points_to_predict['predicted_climate'].map(inverse_climate_mapping)

# Exibir tabela no Jupyter
points_to_predict

# Opcional: Exibir a tabela gráfica no console e exportar para Excel/CSV se necessário
points_to_predict.style.set_caption("Previsão de Climas para Pontos Específicos").background_gradient(cmap="coolwarm")

# Se quiser salvar em CSV:
# points_to_predict.to_csv('dataset/predicted_points.csv', index=False)


,age,basin,lat,long,predicted_climate
0,145,camamu-almada,-13.830000,-38.500000,h
1,145,camamu-almada,-13.830000,-38.500000,h
2,145,camamu-almada,-13.830000,-38.500000,h
3,140,araripe,-7.500000,-39.500000,s
4,140,Camamu-almada,-13.830000,-38.500000,h
5,140,Camamu-almada,-13.830000,-38.500000,h


In [21]:
import folium
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# Ler dados de clima a partir do arquivo CSV
climate_data = pd.read_csv('dataset/points.csv')

# Mapeamento de climas
climate_mapping = {
    'h': 1,  # Exemplo: "h" para 1
    's': 2,  # Exemplo: "s" para 2
    't': 3   # Exemplo: "t" para 3
}

# Aplicando mapeamento aos dados de clima
climate_data['climate'] = climate_data['climate'].map(climate_mapping)

# Remover linhas com NaN na coluna 'climate'
climate_data = climate_data.dropna(subset=['climate'])

# Separando features (X) e alvo (y)
X = climate_data[['lat', 'long', 'age']]
y = climate_data['climate']

# Normalizando os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Treinando o modelo KNN (k=3)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_scaled, y)

# Função para prever o clima em uma nova posição
def prever_clima(lat, long, age):
    ponto_hipotetico = pd.DataFrame({
        'lat': [lat],
        'long': [long],
        'age': [age]
    })

    # Normalizando o ponto
    ponto_hipotetico_scaled = scaler.transform(ponto_hipotetico)

    # Fazendo a previsão
    clima_predito = knn.predict(ponto_hipotetico_scaled)
    return clima_predito[0]

# Mapeamento inverso para converter números de volta para string
inverse_climate_mapping = {v: k for k, v in climate_mapping.items()}

# Ler o CSV com os pontos a serem previstos
points_to_predict = pd.read_csv('dataset/points_to_predict.csv')

# Aplicar previsões em cada linha do CSV
points_to_predict['predicted_climate'] = points_to_predict.apply(
    lambda row: prever_clima(row['lat'], row['long'], row['age']), axis=1
)

# Converter valores numéricos de volta para strings de clima
points_to_predict['predicted_climate'] = points_to_predict['predicted_climate'].map(inverse_climate_mapping)

# Criar mapa com folium
mapa = folium.Map(location=[-14, -38], zoom_start=5)

# Função para definir ícones com base no clima
def get_icon(climate, predicted=False):
    color = 'blue'
    icon = 'circle'

    if climate == 'h':  # Clima h
        color = 'red'
        icon = 'info-sign' if predicted else 'circle'
    elif climate == 's':  # Clima s
        color = 'blue'
        icon = 'cloud' if predicted else 'circle'
    elif climate == 't':  # Clima tropical
        color = 'green'
        icon = 'leaf' if predicted else 'circle'

    return folium.Icon(color=color, icon=icon, prefix='glyphicon')

# Adicionar pontos originais ao mapa
for _, row in climate_data.iterrows():
    folium.Marker(
        location=[row['lat'], row['long']],
        popup=f"Clima Original: {inverse_climate_mapping[row['climate']]}",
        icon=get_icon(inverse_climate_mapping[row['climate']])
    ).add_to(mapa)

# Adicionar pontos previstos ao mapa
for _, row in points_to_predict.iterrows():
    folium.Marker(
        location=[row['lat'], row['long']],
        popup=f"Clima Previsto: {row['predicted_climate']}",
        icon=get_icon(row['predicted_climate'], predicted=True)
    ).add_to(mapa)

# Exibir o mapa
mapa
